<a href="https://colab.research.google.com/github/RV05/Classification-of-disasters-using-modified-architrctures/blob/main/effecientnet_disaster_Adam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/SCAAI_Rohit_volety_Problem_Statement_3/deep learning"

/content/drive/MyDrive/SCAAI_Rohit_volety_Problem_Statement_3/deep learning


In [ ]:
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
import config
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19


from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import GlobalAveragePooling2D

from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
# ap = argparse.ArgumentParser()
# ap.add_argument("-p", "--plot", type=str, default="plot.png",
# 	help="path to output loss/accuracy plot")
# args = vars(ap.parse_args())
# # determine the total number of image paths in training, validation,
# # and testing directories


In [ ]:
totalTrain = len(list(paths.list_images(config.TRAIN_PATH)))
totalVal = len(list(paths.list_images(config.VAL_PATH)))
totalTest = len(list(paths.list_images(config.TEST_PATH)))
trainAug = ImageDataGenerator(
	# rotation_range=25,
	# zoom_range=0.1,
	# width_shift_range=0.1,
	# height_shift_range=0.1,
	# shear_range=0.2,
	# horizontal_flip=True,
  # vertical_flip=True,
	# fill_mode="nearest"
  )

In [ ]:
# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

In [ ]:
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
# mean = np.array([123.68, 116.779, 103.939], dtype="float32")
# trainAug.mean = mean
# valAug.mean = mean

In [ ]:
trainGen = trainAug.flow_from_directory(
	config.TRAIN_PATH,
	class_mode="categorical",
	target_size=(200, 200),
	color_mode="rgb",
	shuffle=True,
	batch_size=config.BS)
# initialize the validation generator
valGen = valAug.flow_from_directory(
	config.VAL_PATH,
	class_mode="categorical",
	target_size=(200, 200),
	color_mode="rgb",
	shuffle=False,
	batch_size=config.BS)
# initialize the testing generator
testGen = valAug.flow_from_directory(
	config.TEST_PATH,
	class_mode="categorical",
	target_size=(200, 200),
	color_mode="rgb",
	shuffle=False,
	batch_size=config.BS)

Found 3309 images belonging to 6 classes.
Found 31 images belonging to 6 classes.
Found 1022 images belonging to 6 classes.


In [ ]:
print("[INFO] preparing model...")
baseModel = EfficientNetB7(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(200, 200, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D()(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dense(256, activation="relu")(headModel)

headModel = Dropout(0.5)(headModel)
headModel = Dense(len(config.CLASSES), activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in baseModel.layers:
	layer.trainable = False
# compile the model
opt = Adam(lr=config.INIT_LR, decay=config.INIT_LR / config.NUM_EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the model

[INFO] preparing model...
258088960/258076736 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:

# load the ResNet-50 network, ensuring the head FC layer sets are left
# off

print("[INFO] training model...")
H = model.fit_generator(
	trainGen,
	steps_per_epoch=totalTrain // config.BS,
	validation_data=valGen,
	validation_steps=totalVal // config.BS,
	epochs=config.NUM_EPOCHS)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


[INFO] training model...
Epoch 1/200
413/413 [==============================] - 895s 2s/step - loss: 0.8266 - accuracy: 0.7089 - val_loss: 0.7231 - val_accuracy: 0.8333
Epoch 2/200
413/413 [==============================] - 42s 102ms/step - loss: 0.3968 - accuracy: 0.8667 - val_loss: 1.1071 - val_accuracy: 0.7083
Epoch 3/200
413/413 [==============================] - 43s 104ms/step - loss: 0.2523 - accuracy: 0.9143 - val_loss: 1.1450 - val_accuracy: 0.7500
Epoch 4/200
413/413 [==============================] - 43s 105ms/step - loss: 0.1857 - accuracy: 0.9349 - val_loss: 0.9283 - val_accuracy: 0.7500
Epoch 5/200
413/413 [==============================] - 44s 105ms/step - loss: 0.1367 - accuracy: 0.9549 - val_loss: 1.1473 - val_accuracy: 0.7083
Epoch 6/200
413/413 [==============================] - 44s 106ms/step - loss: 0.0965 - accuracy: 0.9636 - val_loss: 1.1047 - val_accuracy: 0.7917
Epoch 7/200
413/413 [==============================] - 44s 106ms/step - loss: 0.0994 - accuracy: 0.96

In [ ]:
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(totalTest // config.BS) + 1)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))
# serialize the model to disk
print("[INFO] saving model...")
model.save(config.MODEL_PATH, save_format="h5")
N = config.NUM_EPOCHS

[INFO] evaluating network...


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


              precision    recall  f1-score   support

        fire       0.92      0.82      0.87       200
       human       0.82      0.88      0.85        42
       infra       0.46      0.82      0.59       210
        land       0.68      0.60      0.64       148
    nodamage       0.40      0.07      0.11       211
       water       0.69      0.84      0.76       211

    accuracy                           0.64      1022
   macro avg       0.66      0.67      0.64      1022
weighted avg       0.63      0.64      0.60      1022

[INFO] saving model...


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("effecientnet disaster_Adam_plot.png")

In [ ]:
H.history

In [ ]:
from tensorflow.keras.preprocessing import image
import tensorflow as tf

import os
dir_path='/content/drive/MyDrive/SCAAI_Rohit_volety_Problem_Statement_3/deep learning/disaster/training/human'
model=tf.keras.models.load_model("/content/drive/MyDrive/SCAAI_Rohit_volety_Problem_Statement_3/deep learning/disaster.model")
for i in os.listdir(dir_path):
  img=image.load_img(dir_path+'//'+i,target_size=(200,200))
  plt.imshow(img)
  plt.show()
  X=image.img_to_array(img)
  X=np.expand_dims(X,axis=0)
  images=np.vstack([X])
  val=model.predict(images)
  print(val)

[[5.8719549e-37 1.0000000e+00 1.7749625e-24 8.4715123e-34 0.0000000e+00
  1.6505122e-33]]
[[2.1110801e-23 1.0000000e+00 5.1492648e-17 7.3332938e-21 2.1149735e-23
  2.3988142e-20]]
[[0.0000000e+00 1.0000000e+00 8.8340455e-35 0.0000000e+00 0.0000000e+00
  0.0000000e+00]]
[[8.0834020e-12 1.0000000e+00 1.7948553e-08 2.9935759e-10 3.0806564e-12
  4.5450846e-10]]
[[2.7012701e-28 1.0000000e+00 2.3428782e-21 1.1564394e-26 5.2462426e-31
  1.9086257e-26]]
[[0. 1. 0. 0. 0. 0.]]
[[3.6842388e-07 9.9989498e-01 7.2184754e-05 3.1510470e-05 4.0660321e-08
  9.4531634e-07]]
[[0.0000000e+00 1.0000000e+00 2.4161914e-34 0.0000000e+00 0.0000000e+00
  0.0000000e+00]]
[[0. 1. 0. 0. 0. 0.]]
[[1.2163773e-15 1.0000000e+00 3.0667124e-10 5.4054980e-14 7.8933750e-17
  1.3279307e-13]]
[[1.0556992e-22 1.0000000e+00 3.1145153e-16 1.8179249e-20 2.2051709e-24
  1.4044352e-20]]
[[3.3110684e-27 1.0000000e+00 2.4481816e-19 2.9832148e-24 5.9925467e-29
  8.9149707e-24]]
[[0.0000000e+00 1.0000000e+00 1.1011451e-32 0.0000000e+0

KeyboardInterrupt: ignored